In [68]:
import requests
import os
from dotenv import load_dotenv, dotenv_values
from array import array
from PIL import Image
import sys
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
import pandas as pd
import numpy as np
import regex as re

from PIL import Image
import requests
from io import BytesIO

In [2]:
load_dotenv()

True

In [3]:
url = "https://invoiceocrp3.azurewebsites.net/invoices"

headers = {
    "accept": "application/json"
}

In [4]:
data = []

In [5]:
response = requests.get(url, headers=headers)
data = response.json()
data = data.get('invoices')

In [6]:
lastDate = data[-1]['dt']
flag = False
while not flag : 
    urldata  = f"https://invoiceocrp3.azurewebsites.net/invoices?start_date={lastDate}"
    responseData = requests.get(urldata, headers=headers)
    if responseData.status_code == 200:
            adddata = responseData.json()
            adddata = adddata.get('invoices')
            if len(adddata) == 0:
                flag = True
            else:
                data.extend(adddata)
                lastDate = data[-1]['dt']
                print(lastDate,data[-1])
    else :
        print(f"Erreur: {response.status_code} - {response.reason}") 
        flag = True

2020-10-18 05:00:00 {'no': 'FAC_2020_0823-2487', 'dt': '2020-10-18 05:00:00'}
2021-07-24 00:09:00 {'no': 'FAC_2021_0641-519400', 'dt': '2021-07-24 00:09:00'}
2022-05-26 23:53:00 {'no': 'FAC_2022_0474-6057400', 'dt': '2022-05-26 23:53:00'}
2023-03-28 16:10:00 {'no': 'FAC_2023_0295-359724', 'dt': '2023-03-28 16:10:00'}
2024-02-08 06:03:00 {'no': 'FAC_2024_0122-4602818', 'dt': '2024-02-08 06:03:00'}
2024-03-27 07:04:00 {'no': 'FAC_2024_0297-79250', 'dt': '2024-03-27 07:04:00'}


In [7]:
print(data[5]['no'])
print(len(data))

FAC_2019_0005-1869518
6174


In [73]:
read_image_url = f"https://invoiceocrp3.azurewebsites.net/invoices/{data[5]['no']}"

response = requests.get(read_image_url)
img = Image.open(BytesIO(response.content))

In [76]:
print(img)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=850x1100 at 0x19E93FC5C10>


TESSERACT

In [74]:

tessocr = dotenv_values(".env")["PATH_TESSERACT"]
   

In [78]:
dataset = data[:10]
print(dataset)

[{'no': 'FAC_2019_0002-521208', 'dt': '2019-01-01 03:29:00'}, {'no': 'FAC_2019_0001-112650', 'dt': '2019-01-01 08:21:00'}, {'no': 'FAC_2019_0003-4174848', 'dt': '2019-01-02 01:05:00'}, {'no': 'FAC_2019_0004-46050', 'dt': '2019-01-02 07:27:00'}, {'no': 'FAC_2019_0006-6410304', 'dt': '2019-01-03 14:19:00'}, {'no': 'FAC_2019_0005-1869518', 'dt': '2019-01-03 22:10:00'}, {'no': 'FAC_2019_0007-2747871', 'dt': '2019-01-04 00:13:00'}, {'no': 'FAC_2019_0009-2407968', 'dt': '2019-01-04 09:29:00'}, {'no': 'FAC_2019_0008-2031855', 'dt': '2019-01-04 23:49:00'}, {'no': 'FAC_2019_0011-585056', 'dt': '2019-01-05 03:18:00'}]


In [81]:
import pytesseract

for row in range(len(dataset)) :

    read_image_url = f"https://invoiceocrp3.azurewebsites.net/invoices/{dataset[row]['no']}"

    response = requests.get(read_image_url)
    img = Image.open(BytesIO(response.content))
    pytesseract.pytesseract.tesseract_cmd = tessocr

    text = pytesseract.image_to_string(img)

    print(text)
    print("////////////////////////////////")

INVOICE FAC_2019_0002
Issue date 2019-01-01 03:29:00
Bill to Sébastien Jean-Vasseur

Esse reiciendis facere aperiam.

Voluptatum delectus dolore enim.

Autem rem cum architecto.
Rien chaise seconde ciel.

TOTAL

Raaw

wm KX

-80
44
+98
+23

Euro
Euro
Euro
Euro


////////////////////////////////
INVOICE FAC_2019_0001
Issue date 2019-01-01 08:21:00

Bill to Sarah Smith

0496 Brianna Crossing
rabitha, RI 41854

Celui travail élément apporter.
Ad illo adipisci quaerat.

Exercitationem alias dignissimos labore.

Mention back sound center.
Economic everybody north three.
Seulement derriére faute tard.
Whole administration rich final.

TOTAL

BENNO!
mM KM KWOK

98.
63.
15.
68.
11.
89.
91.

Euro
Euro
Euro
Euro
Euro
Euro
Euro

Euro

////////////////////////////////
INVOICE FAC_2019_0003
Issue date 2019-01-02 01:05:00
Bill to Orlando Ovadia

bert

52.90 Euro
59.23 Euro

TOTAL 323.73 Euro

Town seven degree that.

5 x
Accent discussion suivre lévre. ix

////////////////////////////////
INVOICE FA

EASY OCR

In [83]:
import easyocr

reader = easyocr.Reader(['fr'],gpu = False)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [87]:
result = reader.readtext(read_image_url)

Progress: |███████████████████████████████████████████████████| 103.2% Complete

In [90]:
pd.DataFrame(result, columns=['bbox','text','conf'])

,bbox,text,conf
0,"[[16, 18], [302, 18], [302, 48], [16, 48]]",INVOICE FAC_2019_0011,0.770947
1,"[[16, 48], [312, 48], [312, 72], [16, 72]]",Issue date 2019-01-05 03.18.00,0.767960
2,"[[18, 77], [200, 77], [200, 105], [18, 105]]",Bill to Charles Guyot,0.997382
3,"[[19, 121], [165, 121], [165, 133], [19, 133]]",Address 754 Larson Harbor,0.788046
4,"[[19, 133], [205, 133], [205, 147], [19, 147]]",South Alexanderchester. MI 60072,0.627906
5,"[[38, 198], [288, 198], [288, 224], [38, 224]]",Portier cher courage énorme.,0.871925
6,"[[573, 201], [599, 201], [599, 219], [573, 219]]",45,0.999998
7,"[[609, 203], [633, 203], [633, 219], [609, 219]]",98,0.999993
8,"[[643, 203], [693, 203], [693, 219], [643, 219]]",Euro,0.981539
9,"[[38, 218], [208, 218], [208, 242], [38, 242]]",Base per sister loss.,0.883240


KERAS 

In [3]:
!pip uninstall -q keras-ocr

^C


In [10]:
import keras_ocr

pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\RED94\.keras-ocr\craft_mlt_25k.h5


ValueError: Unrecognized keyword arguments passed to Dense: {'weights': [array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
   

In [6]:
pipeline.recognize(read_image_url)

NameError: name 'pipeline' is not defined